In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import h5py

from sklearn import model_selection, feature_selection, utils, ensemble, linear_model, metrics

In [ ]:
print("Import data")

X = pd.read_hdf('drive/My Drive/data_window_botnet3.h5', key='data')
X.reset_index(drop=True, inplace=True)

X2 = pd.read_hdf('drive/My Drive/data_window3_botnet3(1).h5', key='data')
X2.reset_index(drop=True, inplace=True)

X = X.join(X2)

X.drop('window_id', axis=1, inplace=True)

y = X['Label_<lambda>']
X.drop('Label_<lambda>', axis=1, inplace=True)

labels = np.load("drive/My Drive/data_window_botnet3_labels.npy",allow_pickle=True)

Import data


In [ ]:
print(X.columns.values)
print(labels)
print(np.where(labels == 'flow=From-Botne')[0][0])

y_bin6 = y==np.where(labels == 'flow=From-Botne')[0][0]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_bin6, test_size=0.33, random_state=123456)

print(X_train.shape)
X_train_new, y_train_new = utils.resample(X_train, y_train, n_samples=X_train.shape[0]*20, random_state=123456)
print("X",np.unique(X, return_counts=True))
#print("X_train", np.unique(x_train, return_counts=True))
#print("X_test", np.unique(x_test, return_counts=True))
print("y", np.unique(y, return_counts=True))
print("y_train", np.unique(y_train_new, return_counts=True))
print("y_test", np.unique(y_test, return_counts=True))

['counts' 'Sport_nunique' 'DstAddr_nunique' 'Dport_nunique' 'Dur_sum'
 'Dur_mean' 'Dur_std' 'Dur_max' 'Dur_median' 'TotBytes_sum'
 'TotBytes_mean' 'TotBytes_std' 'TotBytes_max' 'TotBytes_median'
 'SrcBytes_sum' 'SrcBytes_mean' 'SrcBytes_std' 'SrcBytes_max'
 'SrcBytes_median' 'Sport_RU' 'DstAddr_RU' 'Dport_RU']
['flow=Background' 'flow=From-Norma' 'flow=Normal-V44' 'flow=From-Backg'
 'flow=To-Backgro' 'flow=To-Normal-' 'flow=From-Botne']
6
(1356115, 22)
X (array([ -5.53569439,  -5.52772763,  -5.52758795, ..., 718.47440863,
       734.57982153, 848.21976477]), array([50265,     1,     1, ...,     1,     1,     1]))
y (array([0, 1, 2, 3, 4, 5, 6]), array([1903499,    5483,       5,     986,  112392,     679,    1009]))
y_train (array([False,  True]), array([27109184,    13116]))
y_test (array([False,  True]), array([667591,    347]))


In [ ]:
## Embedded Method
print("Random Forest Classifier")

clf = ensemble.RandomForestClassifier(n_estimators=100, random_state=123456, verbose=1, class_weight=None)
clf.fit(X_train_new, y_train_new)

Random Forest Classifier


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 113.4min finished


RandomForestClassifier(random_state=123456, verbose=1)

In [ ]:
print("Train")
y_pred_train = clf.predict(X_train_new)
print("accuracy score = ", metrics.balanced_accuracy_score(y_train_new, y_pred_train))
precision, recall, fbeta_score, support = metrics.precision_recall_fscore_support(y_train_new, y_pred_train)
print("precision = ", precision[1])
print("recall = ", recall[1])
print("fbeta_score = ", fbeta_score[1])
print("support = ", support[1])

Train


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  3.2min finished


accuracy score =  0.9929094236047575
precision =  1.0
recall =  0.9858188472095151
fbeta_score =  0.9928587882976273
support =  13116


In [ ]:
print("Test")
y_pred_test = clf.predict(X_test)
print("accuracy score = ", metrics.balanced_accuracy_score(y_test, y_pred_test))
precision, recall, fbeta_score, support = metrics.precision_recall_fscore_support(y_test, y_pred_test)
print("precision = ", precision[1])
print("recall = ", recall[1])
print("fbeta_score = ", fbeta_score[1])
print("support = ", support[1])

Test


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    4.7s finished


accuracy score =  0.9827089337175792
precision =  1.0
recall =  0.9654178674351584
fbeta_score =  0.9824046920821115
support =  347
